<div style="text-align: center;">
<img src="https://assets-global.website-files.com/62b9d45fb3f64842a96c9686/62d84db4aeb2f6552f3a2f78_Quantinuum%20Logo__horizontal%20blue.svg" width="200" height="200" /></div>

# Advanced Compilation Options with H-Series

* [H-Series Hardware Compilation](#H-Series-Hardware-Compilation)
  * [Compiling to H-Series Hardware Native Gates](#Compiling-to-H-Series-Hardware-Native-Gates)
  * [The SU(4) Gate](#The-SU(4)-Gate)
  * [Controlling H-Series Hardware Compiler Optimizations](#Controlling-H-Series-Hardware-Compiler-Optimizations)
    * [Circuits written in any gate set](#Circuits-written-in-any-gate-set)
    * [Circuits written in the hardware's native gate set](#Circuits-written-in-the-hardware's-native-gate-set)
* [Examples](#Examples)
    * [1. Benchmarking Circuit in non-native gate set](#1.-Benchmarking-Circuit-in-non-native-gate-set)
    * [2. QFT Circuits in native gate set](#2.-QFT-Circuits-in-native-gate-set)
    * [3. Circuit using $SU(4)$](#Circuit-using-$SU(4)$)

## H-Series Hardware Compilation

Native gates are gates on a quantum computer that the hardware physically executes. Different quantum computers may have different gates that are physically executed on the hardware. Writing a gate in a quantum circuit doesn't guarantee its physical execution on the device. For instance, on H-Series quantum computers, a Hadamard gate written in the circuit is not the actual gate executed. When users submit circuits using a Hadamard gate, the gate is translated into a $U1q$ gate followed by a $Rz$ gate, which the ion trap device physically executes. See the *System Model H1 Product Data Sheet* on the [System Model H1](https://www.quantinuum.com/hardware/h1) page or the *System Model H2 Product Data Sheet* on the [System Model H2](https://www.quantinuum.com/hardware/h2) page for a listing of the H-Series' hardware native gates. 

The H-Series hardware compiler handles the translation from circuits users submit to the native gates run on hardware. In the H-Series Quantum Charge-Coupled Device (QCCD) architecture, the hardware compilation includes the assignment of which physical qubit corresponds to which qubit in a circuit as well as how qubits will be transported around the device. Since transport, as well as gating, incurs a small amount of error with each operation, the H-Series compiler aims to minimize the number of gates that need to be executed. 

**Insert illustration of job --> TKET --> hardware compiler --> hardware.**

### Compiling to H-Series Hardware Native Gates

On the Quantinuum H-Series devices, there are different native two-qubit gates available. The default native two-qubit gates include a fully entangling two-qubit gate, $ZZ()$, and an arbitrary angle ZZ gate, $Rzz(\theta)$. Another native gate available on the hardware is the $SU(4)$ gate.

By default, the hardware compiler compiles to the $ZZ()$ or $Rzz(\theta)$ gate. Currently, only one native gate can be specified at a time. This ensures everything aligns in the global operations of the circuit. 

If users would like to use the $SU(4)$ gate and not have the circuit rebased to $ZZ()$ or $Rzz(\theta)$ by the hardware compiler, they need to specify the $SU(4)$ gate using the `nativetq` option. The `nativetq` option is available to override the hardware stack's default two-qubit gate and use the supplied gate instead.

* `nativetq`: override the stack's default native two-qubit gate and use the supplied gate as the gate instead
  * `ZZ`: compile circuit to the $ZZ$ gate
  * `RZZ`: compile circuit to the $Rzz(\theta)$ gate
  * `Rxxyyzz`: compile circuit to the $SU(4)$ gate. This is also known as `Optype.TK2` within `tket`.

### The $SU(4)$ Gate
The $SU(4)$ gate is available in `tket` as [`OpType.TK2`](https://tket.quantinuum.com/api-docs/circuit_class.html#pytket.circuit.Circuit.TK2). The gate is a combination of `OpType.XXPhase`, `OpType.YYPhase` and `OpType.ZZPhase`, and requires three angles as input, $\alpha$, $\beta$ and $\gamma$. The definition of the gate is provided below:
$$\begin{equation} \textrm{TK2}(\alpha, \beta, \gamma) = e^{-\frac{1}{2} j \pi \alpha (\hat{X} \bigotimes \hat{X})} \quad e^{-\frac{1}{2} j \pi \beta (\hat{Y} \bigotimes \hat{Y})} \quad e^{-\frac{1}{2} \pi \gamma (\hat{Z} \bigotimes \hat{Z})} \end{equation}$$
Within `QuantinuumAPI`, this gate is known as `Rxxyyzz`. The gate can be used as follows within `tket`.

In [2]:
from pytket.circuit.display import render_circuit_jupyter
from pytket.circuit import Circuit
from sympy import Symbol

symbols = [Symbol("a"), Symbol("b"), Symbol("c")]
circuit = Circuit(2)
circuit.TK2(*symbols, *circuit.qubits)
render_circuit_jupyter(circuit)

This circuits can be converted to qasm using the [`circuit_to_qasm_str`](https://tket.quantinuum.com/api-docs/qasm.html#pytket.qasm.circuit_to_qasm_str) function and specifying the header `hqslib1`.

In [6]:
from pytket.qasm.qasm import circuit_to_qasm_str

circuit_to_qasm_str(circuit, header="hqslib1")

'OPENQASM 2.0;\ninclude "hqslib1.inc";\n\nqreg q[2];\nRxxyyzz((a)*pi,(b)*pi,(c)*pi) q[0],q[1];\n'

### Controlling H-Series Hardware Compiler Optimizations

Users have the option of submitting circuits using whichever quantum gate set they desire. Users do not need to think about which physical gates will be executed or how physical qubits will move around the device since the hardware compiler handles this. In certain cases, however, users may want to know that the circuit they submit is going to be run on the device exactly as they write it. For example, when running benchmarking circuits users may want circuits to be executed exactly as specified in the circuit, even if its not the most optimal in total number of two-qubit gates. 

Within the Quantinuum stack, the ability to control levels of optimizations and control over what is executed on the hardware is provided between 4 different job submission parameters in the API.

Users may choose to apply `tket` optimizations remotely to their circuit and control this with `tket-opt-level`, even if the circuit is written in the native gate set since further reductions in the number of quantum gates may be found, which will improve results. If a user submits a circuit in the native gate set of the hardware, `tket` optimizations will run on that gate set to identify further optimizations.

* `tket-opt-level`: the `tket` optimization level to apply (default: `2`), with `tket` optimizations turned on, the hardware compiler will provide further gate combination logic as makes sense for ions and transport
  * `2`: powerful optimizations, can use approximate methods, compilation can be expensive
  * `1`: basic optimization, compiles quickly
  * `0`: rebase the circuit with `tket`
  * `null`: rebase the circuit without `tket`, using the hardware compiler only

There are two ways to think about using these options:
1. [Circuits written in any gate set](#Circuits-written-in-any-gate-set)
2. [Circuits written in the hardware's native gate set](#Circuits-written-in-the-hardware's-native-gate-set)

#### Circuits written in any gate set

Users are free to submit circuits written with any gate set, not just the native gate set of the hardware. In this case, the options for control over what optimizations are applied are given at the TKET level. TKET will rebase the circuit to the native gate set it believes is most optimal and the hardware compiler will handle further optimizations of gate combinations as it applies to transport and ion assignment. We recommend this for the majority of use cases.

#### Circuits written in the hardware's native gate set

For circuits that are written only using gates in the hardware's native gate set, various levels of control are provided for what optimizations will be performed in the stack.

In cases where a user's circuit is written in the native gate set, but the user desires more control over what types, if any, additional optimizations are performed on their circuit, the following options are available. 

*Note:* To use these options, the `tket-opt-level` must be set to `null` when submitting. 

* `no-opt`: turns off all `tket` optimizations *and* all hardware compiler gate combination logic. If more than 1 native gate is used in the circuit, the circuit will be rebased to 1 native gate since that is all the hardware performs at this point, as noted above, but no further gate combination logic will occur. (default: `False`)
    * The job will fail if `no-opt` is set to `True` and the circuit contains non-native gates. 
* `noreduce`: turns off all `tket` optimizations, all hardware compiler gate combination logic, and requires exact 1:1 correspondence of two-qubit gates with gates on the system. This requires the circuit be submitted using one 1 of the native two-qubit gates on the system, otherwise an error will be returned. (default: `False`)

Note that `tket-opt-level` set to `null`, `no-opt` set to `True`, and `noreduce` set to `True` all disable `tket` optimizations, but `no-opt` and `noreduce` also turn off all hardware gate combination logic.

**Create plot of quantum circuit -> tket opt or no-opt or nativetq or noreduce**

## Examples



Now we illustrate the above options with a few examples. First we import the functions we need in pytket.   

In [6]:
import pandas as pd

from pytket import Circuit
from pytket.circuit.display import render_circuit_jupyter

from pytket.extensions.quantinuum import QuantinuumBackend

### 1. Benchmarking Circuit in non-native gate set

In [ ]:
circuit = Circuit(2)
 
circuit.X(0)
circuit.CX(0,1)
circuit.X(0)
circuit.X(1)
 
circuit.add_barrier([0,1])
 
circuit.Z(0)
circuit.CX(0,1)
circuit.Z(0)
 
circuit.add_barrier([0,1])
 
circuit.Y(1)
circuit.CX(0,1)
circuit.Z(0)
circuit.X(1)
 
circuit.add_barrier([0,1])
 
circuit.Z(1)
circuit.CX(0,1)
circuit.Z(0)
circuit.Z(1)
 
render_circuit_jupyter(circuit)

In [ ]:
machine = 'H1-1E'
backend = QuantinuumBackend(device_name=machine)
backend.login()

Essentially if users don't submit in native gate set, their circuit will go through TKET, so its a matter of deciding what TKET optimization level to use.

In [ ]:
compiled_circuit_0 = backend.get_compiled_circuit(circuit, optimisation_level=0)
compiled_circuit_1 = backend.get_compiled_circuit(circuit, optimisation_level=1)
compiled_circuit_2 = backend.get_compiled_circuit(circuit, optimisation_level=2)

In [ ]:
print(compiled_circuit_0.n_2qb_gates(), compiled_circuit_0.n_1qb_gates(), compiled_circuit_0.n_gates)
print(compiled_circuit_1.n_2qb_gates(), compiled_circuit_1.n_1qb_gates(), compiled_circuit_1.n_gates)
print(compiled_circuit_2.n_2qb_gates(), compiled_circuit_2.n_1qb_gates(), compiled_circuit_2.n_gates)

In [ ]:
n_shots = 100
backend.cost(compiled_circuit, n_shots=n_shots, syntax_checker='H1-1SC')

In [ ]:
handle = backend.process_circuit(compiled_circuit, 
                                 n_shots=n_shots,
                                options={'tket-opt-level': None})
print(handle)

In [ ]:
result = backend.get_result(handle)

In [ ]:
import json

with open('pytket_example.json', 'w') as file:
    json.dump(result.to_dict(), file)

In [ ]:
result = backend.get_result(handle)
print(result.get_distribution())
print(result.get_counts())

### 2. QFT Circuits in native gate set

In [2]:
def control_phase(circ, theta, q0, q1, arbZZ=False):
    """circuit gadget for performing controlled-[1 0;0 e^i theta]
    Inputs:
        circ: pytket Circuit object
        theta: Z rotation angle (in units of pi)
        q0: control qubit
        q1: target qubit
        arbZZ (bool): enables arbitrary angle RZZ gate
    """
    if arbZZ == False:
        # decompose into CNOTs
        circ.Rz(theta / 2, q1)
        circ.CX(q0, q1)
        circ.Rz(-theta / 2, q1)
        circ.CX(q0, q1)
        circ.Rz(theta / 2, q0)
    elif arbZZ == True:
        circ.Rz(theta / 2, q0)
        circ.Rz(theta / 2, q1)
        if theta > 0:
            circ.X(q0)
            circ.ZZPhase(theta / 2, q0, q1)
            circ.X(q0)
        elif theta <= 0:
            circ.ZZPhase(-theta / 2, q0, q1)

In [3]:
def QFT(n, **kwargs):
    """
    Function to implement the Quantum Fourier Transform
    n : number of qubits
    approx: if set to integer k, then sets that largest
                 value of pi/2**k occuring in controlled rotation
    returns circ: pytket Circuit object
    """

    # optional keyword arguments
    arbZZ = kwargs.get("arbZZ", False)
    approx = kwargs.get("approx", None)

    # initialize
    circ_name = "QFT-arbZZ" if arbZZ else "QFT-fixed"
    circ = Circuit(n, n, name=circ_name)
    for j in range(n - 1):
        q = n - 1 - j
        circ.H(q)
        for i in range(j + 1):
            if approx == None or approx >= j + 1 - i:
                control_phase(
                    circ, 1 / (2 ** (j + 1 - i)), q - 1, n - 1 - i, arbZZ=arbZZ
                )
    circ.H(0)
    circ.measure_all()
    return circ

In [4]:
n_qubits = 12
qft_fixed = QFT(n_qubits, arbZZ=False)
qft_arbZZ = QFT(n_qubits, arbZZ=True)

In [ ]:
render_circuit_jupyter(qft_fixed)

In [ ]:
render_circuit_jupyter(qft_arbZZ)

In [7]:
machine = 'H1-1E'
backend = QuantinuumBackend(device_name=machine)
backend.login()

Enter your Quantinuum email:  megan.l.kohagen@quantinuum.com
Enter your Quantinuum password:  ········


In [8]:
qft_fixed_circuit_0 = backend.get_compiled_circuit(qft_fixed, optimisation_level=0)
qft_fixed_circuit_1 = backend.get_compiled_circuit(qft_fixed, optimisation_level=1)
qft_fixed_circuit_2 = backend.get_compiled_circuit(qft_fixed, optimisation_level=2)

qft_arbZZ_circuit_0 = backend.get_compiled_circuit(qft_arbZZ, optimisation_level=0)
qft_arbZZ_circuit_1 = backend.get_compiled_circuit(qft_arbZZ, optimisation_level=1)
qft_arbZZ_circuit_2 = backend.get_compiled_circuit(qft_arbZZ, optimisation_level=2)

In [16]:
def circuit_stats(circuit, opt_level, n_shots):
    """ Summarize circuit stats in a list """
    circuit_cost = backend.cost(circuit, n_shots=n_shots, syntax_checker='H1-1SC')
    circuit_row = [circuit.name, opt_level, circuit.n_2qb_gates(), circuit.n_1qb_gates(), circuit.n_gates, circuit_cost]
    return circuit_row

In [18]:
n_shots = 100
qft_data = pd.DataFrame([circuit_stats(qft_fixed_circuit_0, 0, n_shots), circuit_stats(qft_fixed_circuit_1, 1, n_shots),
                         circuit_stats(qft_fixed_circuit_2, 2, n_shots), circuit_stats(qft_arbZZ_circuit_0, 0, n_shots),
                         circuit_stats(qft_arbZZ_circuit_1, 1, n_shots), circuit_stats(qft_arbZZ_circuit_2, 2, n_shots)], 
                        columns=["qft_type", "tket_opt_level", "n_2q_gates", "n_1q_gates", "n_gates", "hqc_cost"])

In [19]:
qft_data.sort_values(by='hqc_cost')

,qft_type,tket_opt_level,n_2q_gates,n_1q_gates,n_gates,hqc_cost
4,QFT-arbZZ,1,66,23,101,21.06
2,QFT-fixed,2,66,73,151,22.06
5,QFT-arbZZ,2,66,85,163,22.30
3,QFT-arbZZ,0,66,288,366,23.48
1,QFT-fixed,1,132,144,288,36.68
0,QFT-fixed,0,132,1014,1158,39.32


In [ ]:
# Default behavior
# handle = backend.process_circuit(compiled_circuit, 
#                                  n_shots=n_shots,
#                                 options={'tket-opt-level': 2,
#                                          'nativetq': 'ZZ',
#                                          'no-opt': False,
#                                          'noreduce': False
#                                         }
#                                 )
# print(handle)

In [ ]:
handle = backend.process_circuit(compiled_circuit, 
                                 n_shots=n_shots,
                                options={'tket-opt-level': None,
                                         'nativetq': '',
                                         'no-opt': True,
                                         'noreduce': True
                                        }
                                )
print(handle)

In [22]:
QuantinuumBackend.two_qubit_gate_set

Because we ran the TKET optimization using `get_compiled_circuit` and we have the circuit we want to run, we turn off further TKET optimizations.

In [ ]:
handle = backend.process_circuit(qft_fixed, 
                                 n_shots=n_shots,
                                options={'tket-opt-level': None})
print(handle)

The arbitrary angle ZZ QFT benefits from the ZZPhase gate, so we specify it as the native two-qubit gate to use using the `nativetq` option.

In [ ]:
handle = backend.process_circuit(qft_arbZZ_circuit_1, 
                                 n_shots=n_shots,
                                 options={'tket-opt-level': None,
                                          'nativetq': 'RZZ'}
                                )
print(handle)

In [ ]:
handle = backend.process_circuit(compiled_circuit, 
                                 n_shots=n_shots,
                                options={'tket-opt-level': None,
                                         'nativetq': 'RZZ',
                                         'no-opt': True
                                        }
                                )
print(handle)

In [ ]:
handle = backend.process_circuit(compiled_circuit, 
                                 n_shots=n_shots,
                                options={'tket-opt-level': None,
                                         'nativetq': 'RZZ',
                                         'no-opt': True,
                                         'noreduce': True
                                        }
                                )
print(handle)

In [ ]:
result = backend.get_result(handle)

In [ ]:
import json

with open('pytket_example.json', 'w') as file:
    json.dump(result.to_dict(), file)

In [ ]:
result = backend.get_result(handle)
print(result.get_distribution())
print(result.get_counts())

### 3. Circuit using $SU(4)$

<div align="center"> &copy; 2024 by Quantinuum. All Rights Reserved. </div>